In [1]:
import os
os.environ['MLFLOW_TRACKING_URI']='https://dagshub.com/researchanalystforapurpose/FACIAL_BIOMETRIC_ATTENDENCE_SYSTEM.mlflow'
os.environ['MLFLOW_TRACKIN_USER_NAME']='researchanalystforapurpose'
os.environ['MLFLOW_TRACKING_PASSWORD']='8e0daaa2b9f63126713e4d435ab226f1e1a19c0c '
# os.environ["HTTP_PROXY"] = "http://172.31.2.4:8080"
# os.environ["HTTPS_PROXY"] = "http://172.31.2.4:8080"
os.environ["NO_PROXY"] = "https://dagshub.com/researchanalystforapurpose/FACIAL_BIOMETRIC_ATTENDENCE_SYSTEM.mlflow"

In [2]:
os.chdir('../')
%pwd


'c:\\Users\\LENOVO\\OneDrive\\Desktop\\complete_end_to_end_project'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class model_evaluation_config:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_path: Path
    all_params: dict
    target_column: str
    mlflow_uri: str
    

In [4]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yml,save_json,create_directories

class model_evaluation_config_manger:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH,schema_file_path=SCHEMA_FILE_PATH):
        self.config=read_yml(config_file_path)
        self.schema=read_yml(schema_file_path)
        self.params=read_yml(params_file_path)

        create_directories([self.config.artifacts_roots])

    def get_model_evaluation_config(self)->model_evaluation_config:
        config=self.config.model_evaluation
        params=self.params.ELASTIC_NET
        schema=self.schema.TARGET_COLUMNS
        create_directories([config.root_dir])

        evaluation_config=model_evaluation_config(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metric_file_path=config.metric_file_path,
            all_params=params,
            mlflow_uri=config.mlflow_uri,
            target_column=schema.NAME
        )

        return evaluation_config
        

In [5]:
import pandas as pd
import numpy as np
import mlflow
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
import joblib
from urllib.parse import urlparse


class model_evaluation:
    def __init__(self,config=model_evaluation_config):
        self.config=config

    def eval_metrcis(self,actual,pred):
        r2_score=r2_score(actual,pred)
        mae=mean_absolute_error(actual,pred)
        root_mean_squared_error=np.sqrt(mean_squared_error(actual,pred))

        return r2_score,mae,root_mean_squared_error    
    
    def log_into_mlflow(self):
        test_data=pd.read_csv(self.config.test_data_path)

        model=joblib.load(self.config.model_path)
        # print(test_data.shape)
        test_x=test_data.drop(columns=self.config.target_column)
        
        test_y=test_data[[self.config.target_column]]
        
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme
        mlflow.set_experiment("your_experiment_name")
        print('tilll')
        with mlflow.start_run():
            pred=model.predict(test_x)
            (r2_score,mae,rmse)=self.eval_metrcis(test_y,pred)
            print('her')
            scores={'rmse':rmse,'mae':mae,'r2_score':r2_score}
            save_json(Path=Path(self.config.metric_file_path),data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric('rmse',rmse)
            mlflow.log_metric('r2_score',r2_score)
            mlflow.log_metric('mae',mae)
            print('hello')

            if tracking_url_type_store!= 'file':
                mlflow.sklearn.log_model(model,'model',registered_model_name='ELASTIC_NET')

            else:
                mlflow.sklearn.log_model(model,'model')    

In [ ]:
try: 
    
    confif_manager=model_evaluation_config_manger()
    config=confif_manager.get_model_evaluation_config()
    model_evaluation=model_evaluation(config)    
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e


[2025-02-02 02:42:01,935: INFO: common: yaml file config\config.yaml loaded succesfully]
[2025-02-02 02:42:01,961: INFO: common: yaml file schema.yaml loaded succesfully]
[2025-02-02 02:42:01,966: INFO: common: yaml file params.yaml loaded succesfully]
[2025-02-02 02:42:01,968: INFO: common: created directory at path artifacts]
[2025-02-02 02:42:01,968: INFO: common: created directory at path artifacts/model_evaluation]
